In [4]:
import torch
import torchvision
from torchvision.datasets import MNIST
from torchvision import transforms

transformer = transforms.Compose([transforms.ToTensor()])
train_set = MNIST(root='./data', download=True, transform=transformer, train=True)

Failed to download (trying next):
HTTP Error 403: Forbidden

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>



RuntimeError: Error downloading train-images-idx3-ubyte.gz